In [1]:
#importing the dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
from flask import Flask, request, jsonify

In [2]:
# Load and preprocess the dataset
def load_and_preprocess_data():
    df = pd.read_csv('/content/car data.csv')

    # Feature Engineering
    df['Age'] = 2024 - df['Year']
    df = df.drop(['Year', 'Car_Name'], axis=1)

    # One-Hot Encoding
    df = pd.get_dummies(df, columns=['Fuel_Type', 'Selling_type', 'Transmission'], drop_first=True)

    # Features and target
    X = df.drop('Selling_Price', axis=1)
    y = df['Selling_Price']

    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test

In [3]:
# Train Random Forest Model
def train_random_forest(X_train, y_train):
    rf = RandomForestRegressor(n_estimators=100, max_depth=20, min_samples_split=5)
    rf.fit(X_train, y_train)
    return rf

In [4]:
# Train Neural Network Model
def train_neural_network(X_train, y_train):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(128, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=100, validation_split=0.2, verbose=0)
    return model

In [5]:
# Evaluate Model
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test) if hasattr(model, 'predict') else model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, r2

In [6]:
# Save models
def save_models(rf, nn_model):
    joblib.dump(rf, 'random_forest_model.pkl')
    nn_model.save('neural_network_model.h5')

In [7]:
def predict():
    data = request.json
    features = [data['Present_Price'], data['Driven_kms'], data['Age'], data['Fuel_Type'], data['Selling_type'], data['Transmission'], data['Owner']]

    rf = joblib.load('random_forest_model.pkl')
    nn_model = tf.keras.models.load_model('neural_network_model.h5')

    # Feature scaling for API prediction
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform([features])

    rf_prediction = rf.predict(features_scaled)
    nn_prediction = nn_model.predict(features_scaled)[0][0]

    return jsonify({'Random_Forest_Prediction': rf_prediction[0], 'Neural_Network_Prediction': nn_prediction})


In [8]:
X_train, X_test, y_train, y_test = load_and_preprocess_data()

rf_model = train_random_forest(X_train, y_train)
nn_model = train_neural_network(X_train, y_train)

rf_mse, rf_r2 = evaluate_model(rf_model, X_test, y_test)
nn_mse, nn_r2 = evaluate_model(nn_model, X_test, y_test)

print(f"Random Forest MSE: {rf_mse}, R-squared: {rf_r2}")
print(f"Neural Network MSE: {nn_mse}, R-squared: {nn_r2}")

save_models(rf_model, nn_model)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


Random Forest MSE: 0.8230215545485504, R-squared: 0.9642717312502482
Neural Network MSE: 0.6089118576596254, R-squared: 0.973566468125726
